# Build a cuisine recommender

In [1]:
import pandas as pd 

In [2]:
data = pd.read_csv('../../data/cleaned_cuisines.csv')
data.head()

,Unnamed: 0,cuisine,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,0,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,indian,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,indian,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [3]:
X = data.iloc[:,2:]
X.head()

,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,artemisia,artichoke,...,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [4]:
y = data[['cuisine']]
y.head()

,cuisine
0,indian
1,indian
2,indian
3,indian
4,indian


In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, classification_report

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [7]:
# model = OneVsRestClassifier(LogisticRegression(solver='liblinear'))
# model.fit(X_train, np.ravel(y_train))

model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(X_train, y_train.values.ravel())


a:\EDUCATION\ML_Starter\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(multi_class='multinomial')

In [8]:
y_pred = model.predict(X_test)

In [9]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
print(f"Accuracy (train): {accuracy * 100:.1f}%")
print(f"Precision (train): {precision * 100:.1f}%")
print(f"{classification_report(y_test,y_pred)}")
print(f"{confusion_matrix(y_test,y_pred)}")

Accuracy (train): 82.6%
Precision (train): 82.7%
              precision    recall  f1-score   support

     chinese       0.78      0.74      0.76       215
      indian       0.90      0.89      0.90       263
    japanese       0.75      0.83      0.79       244
      korean       0.88      0.78      0.83       224
        thai       0.82      0.87      0.84       253

    accuracy                           0.83      1199
   macro avg       0.83      0.82      0.82      1199
weighted avg       0.83      0.83      0.83      1199

[[159   5  27  10  14]
 [  2 235  10   0  16]
 [ 18   1 202  13  10]
 [ 16   4  22 175   7]
 [  9  15   9   1 219]]


In [10]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# initial_type = [('float_input', FloatTensorType([None, 380]))]
# options = {id(model): {'nocl': True, 'zipmap': False}}
# onx = convert_sklearn(model, initial_types=initial_type, options=options)
# with open("./model.onnx", "wb") as f:
#     f.write(onx.SerializeToString())

initial_type = [('float_input', FloatTensorType([None, 380]))]
onx = convert_sklearn(model, initial_types=initial_type)
with open("./model.onnx", "wb") as f:
    f.write(onx.SerializeToString())


In [15]:
import onnx

model = onnx.load("model.onnx")
print("Inputs:")
for i in model.graph.input:
    print(i.name)
print("\n")

print("Outputs:")
for o in model.graph.output:
    print(o.name)
print("\n")

print("Types:")
for o in model.graph.input:
    print(o.type)

Inputs:
float_input


Outputs:
output_label
output_probability


Types:
tensor_type {
  elem_type: 1
  shape {
    dim {
    }
    dim {
      dim_value: 380
    }
  }
}



In [16]:
import onnxruntime as ort
import numpy as np

session = ort.InferenceSession("model.onnx")
input_name = session.get_inputs()[0].name  # Should match JS
output_name = session.get_outputs()[0].name  # Should match JS

# Example input, update as needed
ingredients = np.zeros((1, 380), dtype=np.float32)
inputs = {input_name: ingredients}
results = session.run(None, inputs)
print(f'Predicted cuisine: {results[0][0]}')

Predicted cuisine: thai
